In [52]:
from pyspark.sql import SparkSession
import sys
spark = SparkSession.builder.getOrCreate()
from pymongo import MongoClient
import hdfs
from pywebhdfs.webhdfs import PyWebHdfsClient
from pprint import pprint
import json
from datetime import datetime as dt
import datetime
import pandas as pd
import pytz

In [72]:
hdfs = PyWebHdfsClient(host='hadoop-master',port='9870', user_name='kafka-data')  # your Namenode IP & username here
my_dir = '/user/kafka-data'
data_files = [i["pathSuffix"] for i in hdfs.list_dir(my_dir)["FileStatuses"]["FileStatus"]]
pprint([data_files, len(data_files)], compact=True)

[['data-0.json', 'data-1.json', 'data-10.json', 'data-11.json', 'data-12.json',
  'data-13.json', 'data-14.json', 'data-15.json', 'data-16.json',
  'data-17.json', 'data-18.json', 'data-19.json', 'data-2.json', 'data-20.json',
  'data-21.json', 'data-22.json', 'data-23.json', 'data-24.json',
  'data-25.json', 'data-26.json', 'data-27.json', 'data-28.json',
  'data-29.json', 'data-3.json', 'data-30.json', 'data-31.json', 'data-32.json',
  'data-33.json', 'data-34.json', 'data-35.json', 'data-36.json',
  'data-37.json', 'data-38.json', 'data-39.json', 'data-4.json', 'data-40.json',
  'data-41.json', 'data-42.json', 'data-43.json', 'data-44.json',
  'data-45.json', 'data-46.json', 'data-47.json', 'data-48.json',
  'data-49.json', 'data-5.json', 'data-50.json', 'data-51.json', 'data-52.json',
  'data-53.json', 'data-54.json', 'data-55.json', 'data-56.json',
  'data-57.json', 'data-58.json', 'data-59.json', 'data-6.json', 'data-60.json',
  'data-61.json', 'data-62.json', 'data-63.json', 'da

In [67]:
conn = MongoClient('mongo', 27017)
db = conn.tweets
collection = db.tweet

In [73]:
collection.drop()

for i in data_files:
    df = spark.read.json(f'hdfs://hadoop-master:9000/user/kafka-data/{i}')
    json_data = df.toJSON().collect()
    for i in json_data:
        data = json.loads(i)
        hashtags = [i for i in data["hashtags"]]
        for h in hashtags:
            hours = 2
            hours_added = datetime.timedelta(hours = hours)
            
            timezone = pytz.timezone("Europe/paris")
            new_datetime = dt.fromtimestamp(int(data["timestamp"])/1000)
            new_datetime = new_datetime + hours_added

            uniqued_hashtag = {"hashtag": h, "datetime": new_datetime}
            collection.insert_one(uniqued_hashtag)
        
cursor = collection.find()
for record in cursor:
    print(record)

{'_id': ObjectId('60d5ad32e0a097dedfda8b31'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 39, 878000)}
{'_id': ObjectId('60d5ad32e0a097dedfda8b32'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 86000)}
{'_id': ObjectId('60d5ad32e0a097dedfda8b33'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 374000)}
{'_id': ObjectId('60d5ad32e0a097dedfda8b34'), 'hashtag': 'NazisrausausdenStadien', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 537000)}
{'_id': ObjectId('60d5ad32e0a097dedfda8b35'), 'hashtag': 'HUNGER', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 537000)}
{'_id': ObjectId('60d5ad32e0a097dedfda8b36'), 'hashtag': 'Goretzka', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 537000)}
{'_id': ObjectId('60d5ad32e0a097dedfda8b37'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 23, 40, 537000)}
{'_id': ObjectId('60d5ad32e0a097dedfda8b38'), 'hashtag': '

{'_id': ObjectId('60d5ad33e0a097dedfda9154'), 'hashtag': 'FRA', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 22, 278000)}
{'_id': ObjectId('60d5ad33e0a097dedfda9155'), 'hashtag': 'NED', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 22, 278000)}
{'_id': ObjectId('60d5ad33e0a097dedfda9156'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 22, 285000)}
{'_id': ObjectId('60d5ad33e0a097dedfda9157'), 'hashtag': 'TeamPSG', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 22, 536000)}
{'_id': ObjectId('60d5ad33e0a097dedfda9158'), 'hashtag': 'Pochettino', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 22, 536000)}
{'_id': ObjectId('60d5ad33e0a097dedfda9159'), 'hashtag': 'Wijnaldum', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 22, 536000)}
{'_id': ObjectId('60d5ad33e0a097dedfda915a'), 'hashtag': 'Euro2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 39, 22, 746000)}
{'_id': ObjectId('60d5ad33e0a097dedfda915b'), 'hashtag': 'HBRFanzone', 'datet

{'_id': ObjectId('60d5ad34e0a097dedfda9777'), 'hashtag': 'ManCity', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 53, 124000)}
{'_id': ObjectId('60d5ad34e0a097dedfda9778'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 53, 124000)}
{'_id': ObjectId('60d5ad34e0a097dedfda9779'), 'hashtag': 'VivoAzzurro', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 54, 788000)}
{'_id': ObjectId('60d5ad34e0a097dedfda977a'), 'hashtag': 'Azzurri', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 54, 788000)}
{'_id': ObjectId('60d5ad34e0a097dedfda977b'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 54, 788000)}
{'_id': ObjectId('60d5ad34e0a097dedfda977c'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 55, 287000)}
{'_id': ObjectId('60d5ad34e0a097dedfda977d'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 47, 56, 928000)}
{'_id': ObjectId('60d5ad34e0a097dedfda977e'), 'hashtag': 'ManCity', 

{'_id': ObjectId('60d5ad35e0a097dedfda9d9a'), 'hashtag': 'Euro2021', 'datetime': datetime.datetime(2021, 6, 25, 11, 27, 48, 748000)}
{'_id': ObjectId('60d5ad35e0a097dedfda9d9b'), 'hashtag': 'PCR', 'datetime': datetime.datetime(2021, 6, 25, 11, 27, 48, 748000)}
{'_id': ObjectId('60d5ad35e0a097dedfda9d9c'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 27, 50, 589000)}
{'_id': ObjectId('60d5ad35e0a097dedfda9d9d'), 'hashtag': 'ENG', 'datetime': datetime.datetime(2021, 6, 25, 11, 27, 51, 526000)}
{'_id': ObjectId('60d5ad35e0a097dedfda9d9e'), 'hashtag': 'GER', 'datetime': datetime.datetime(2021, 6, 25, 11, 27, 51, 526000)}
{'_id': ObjectId('60d5ad35e0a097dedfda9d9f'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 11, 27, 51, 785000)}
{'_id': ObjectId('60d5ad35e0a097dedfda9da0'), 'hashtag': 'Goretzka', 'datetime': datetime.datetime(2021, 6, 25, 11, 27, 53, 937000)}
{'_id': ObjectId('60d5ad35e0a097dedfda9da1'), 'hashtag': 'EURO2020', 'datetime': dat

{'_id': ObjectId('60d5ad36e0a097dedfdaa3c2'), 'hashtag': 'ENG', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 52, 487000)}
{'_id': ObjectId('60d5ad36e0a097dedfdaa3c3'), 'hashtag': 'GER', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 52, 487000)}
{'_id': ObjectId('60d5ad36e0a097dedfdaa3c4'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 55, 353000)}
{'_id': ObjectId('60d5ad36e0a097dedfdaa3c5'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 55, 477000)}
{'_id': ObjectId('60d5ad36e0a097dedfdaa3c6'), 'hashtag': 'Mariners', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 56, 978000)}
{'_id': ObjectId('60d5ad36e0a097dedfdaa3c7'), 'hashtag': 'RemoveATK', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 56, 978000)}
{'_id': ObjectId('60d5ad36e0a097dedfdaa3c8'), 'hashtag': 'Heineken', 'datetime': datetime.datetime(2021, 6, 25, 12, 15, 57, 420000)}
{'_id': ObjectId('60d5ad36e0a097dedfdaa3c9'), 'hashtag': 'musulmane', 'datetim